# Prerequisites
Install Theano and Lasagne using the following commands:

```bash
pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt
pip install https://github.com/Lasagne/Lasagne/archive/master.zip
```

Working in a virtual environment is recommended.

# Data preparation

Current code allows to generate geodesic patches from a collection of shapes represented as triangular meshes.
To get started with the pre-processing:
```
git clone https://github.com/jonathanmasci/ShapeNet_data_preparation_toolbox.git
```

The usual processing pipeline is show in ```run_forrest_run.m```. 
We will soon update this preparation stage, so perhaps better to start with our pre-computed dataset, and stay tuned! :-)

## Prepared data

All it is required to train on the FAUST_registration dataset for this demo is available for download at
https://www.dropbox.com/s/aamd98nynkvbcop/EG16_tutorial.tar.bz2?dl=0

# ICNN Toolbox

```bash
git clone https://github.com/jonathanmasci/EG16_tutorial.git
```

![](http://www.people.usi.ch/mascij/EG16_tutorial/shapenet_architecture.png)

In [1]:
import sys
import os
import numpy as np
import scipy.io
import time

import theano
import theano.tensor as T
import theano.sparse as Tsp

import lasagne as L
import lasagne.layers as LL
import lasagne.objectives as LO
from lasagne.layers.normalization import batch_norm

sys.path.append('..')
from icnn import utils_lasagne, dataset, snapshotter

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 5105)


## Data loading

In [2]:
base_path = './dataset/FAUST_registrations/data/diam=200/'

ds = dataset.ClassificationDatasetPatchesMinimal(
    'FAUST_registrations_train.txt', 'FAUST_registrations_test.txt',
    os.path.join(base_path, 'descs', 'shot'),
    os.path.join(base_path, 'patch_aniso', 'alpha=100_nangles=016_ntvals=005_tmin=6.000_tmax=24.000_thresh=99.900_norm=L1'), 
    None, 
    os.path.join(base_path, 'labels'),
    epoch_size=50)

Loading train descs
elapsed time 6.457000
Loading test descs
elapsed time 1.618000
Loading train patches
elapsed time 19.138000
Loading test patches
elapsed time 4.779000
Loading train labels
elapsed time 0.062000
Loading test labels
elapsed time 0.016000


## Network definition

In [3]:
nin = 544
nclasses = 6890
l2_weight = 1e-5

def get_model(inp, patch_op):
    icnn = LL.DenseLayer(inp, 16)
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 16, nrings=5, nrays=16))
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 32, nrings=5, nrays=16))
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 64, nrings=5, nrays=16))
    # TODO: add functional mapping computation and application (maybe instead of the dense layer?)
    ffn = batch_norm(LL.DenseLayer(icnn, 512))
    ffn = LL.DenseLayer(icnn, nclasses, nonlinearity=utils_lasagne.log_softmax)

    return ffn

inp = LL.InputLayer(shape=(None, nin))
patch_op = LL.InputLayer(input_var=Tsp.csc_fmatrix('patch_op'), shape=(None, None))

ffn = get_model(inp, patch_op)

# L.layers.get_output -> theano variable representing network
output = LL.get_output(ffn)
pred = LL.get_output(ffn, deterministic=True)  # in case we use dropout

# target theano variable indicatind the index a vertex should be mapped to wrt the latent space
target = T.ivector('idxs')

# to work with logit predictions, better behaved numerically
cla = utils_lasagne.categorical_crossentropy_logdomain(output, target, nclasses).mean()
acc = LO.categorical_accuracy(pred, target).mean()

# a bit of regularization is commonly used
regL2 = L.regularization.regularize_network_params(ffn, L.regularization.l2)


cost = cla + l2_weight * regL2

## Define the update rule, how to train

In [4]:
params = LL.get_all_params(ffn, trainable=True)
grads = T.grad(cost, params)
# computes the L2 norm of the gradient to better inspect training
grads_norm = T.nlinalg.norm(T.concatenate([g.flatten() for g in grads]), 2)

# Adam turned out to be a very good choice for correspondence
updates = L.updates.adam(grads, params, learning_rate=0.001)

## Compile

In [5]:
funcs = dict()
funcs['train'] = theano.function([inp.input_var, patch_op.input_var, target],
                                 [cost, cla, l2_weight * regL2, grads_norm, acc], updates=updates,
                                 on_unused_input='warn')
funcs['acc_loss'] = theano.function([inp.input_var, patch_op.input_var, target],
                                    [acc, cost], on_unused_input='warn')
funcs['predict'] = theano.function([inp.input_var, patch_op.input_var],
                                   [pred], on_unused_input='warn')

# Training (a bit simplified)

In [6]:
n_epochs = 50
eval_freq = 1

start_time = time.time()
best_trn = 1e5
best_tst = 1e5

kvs = snapshotter.Snapshotter('demo_training.snap')

for it_count in xrange(n_epochs):
    tic = time.time()
    b_l, b_c, b_s, b_r, b_g, b_a = [], [], [], [], [], []
    for x_ in ds.train_iter():
        tmp = funcs['train'](*x_)

        # do some book keeping (store stuff for training curves etc)
        b_l.append(tmp[0])
        b_c.append(tmp[1])
        b_r.append(tmp[2])
        b_g.append(tmp[3])
        b_a.append(tmp[4])
    epoch_cost = np.asarray([np.mean(b_l), np.mean(b_c), np.mean(b_r), np.mean(b_g), np.mean(b_a)])
    print(('[Epoch %03i][trn] cost %9.6f (cla %6.4f, reg %6.4f), |grad| = %.06f, acc = %7.5f %% (%.2fsec)') %
                 (it_count, epoch_cost[0], epoch_cost[1], epoch_cost[2], epoch_cost[3], epoch_cost[4] * 100, 
                  time.time() - tic))

    if np.isnan(epoch_cost[0]):
        print("NaN in the loss function...let's stop here")
        break

    if (it_count % eval_freq) == 0:
        v_c, v_a = [], []
        for x_ in ds.test_iter():
            tmp = funcs['acc_loss'](*x_)
            v_a.append(tmp[0])
            v_c.append(tmp[1])
        test_cost = [np.mean(v_c), np.mean(v_a)]
        print(('           [tst] cost %9.6f, acc = %7.5f %%') % (test_cost[0], test_cost[1] * 100))

        if epoch_cost[0] < best_trn:
            kvs.store('best_train_params', [it_count, LL.get_all_param_values(ffn)])
            best_trn = epoch_cost[0]
        if test_cost[0] < best_tst:
            kvs.store('best_test_params', [it_count, LL.get_all_param_values(ffn)])
            best_tst = test_cost[0]
print("...done training %f" % (time.time() - start_time))

[Epoch 000][trn] cost  8.282597 (cla 8.2768, reg 0.0058), |grad| = 0.275891, acc = 0.16778 % (76.04sec)


ERROR:root:[KVP] Overwriting group best_train_params!


           [tst] cost  7.719462, acc = 0.42888 %


ERROR:root:[KVP] Overwriting group best_test_params!


[Epoch 001][trn] cost  7.242940 (cla 7.2252, reg 0.0177), |grad| = 0.663264, acc = 0.76401 % (77.89sec)


...

           [tst] cost  1.765753, acc = 61.23512 %
[Epoch 048][trn] cost  1.267014 (cla 0.9683, reg 0.2987), |grad| = 2.597066, acc = 71.04644 % (73.17sec)
           [tst] cost  1.834165, acc = 58.81060 %
[Epoch 049][trn] cost  1.218979 (cla 0.9212, reg 0.2977), |grad| = 2.293787, acc = 72.03628 % (73.09sec)


ERROR:root:[KVP] Overwriting group best_train_params!


           [tst] cost  1.746834, acc = 61.26851 %


ERROR:root:[KVP] Overwriting group best_test_params!


...done training 4669.968000


# Test phase
Now that the model is train it is enough to take the fwd function and apply it to new data.

In [7]:
rewrite = True

out_path = './dumps/' 
kvs.load('best_test_params')

print "Saving output to: %s" % out_path

if not os.path.isdir(out_path) or rewrite==True:
    try:
        os.makedirs(out_path)
    except:
        pass
    
    a = []
    for i,d in enumerate(ds.test_iter()):
        fname = os.path.join(out_path, "%s" % ds.test_fnames[i])
        print fname,
        tmp = funcs['predict'](d[0], d[1])[0]
        a.append(np.mean(np.argmax(tmp, axis=1).flatten() == d[2].flatten()))
        scipy.io.savemat(fname, {'desc': tmp})
        print ", Acc: %7.5f %%" % (a[-1] * 100.0)
    print "\nAverage accuracy across all shapes: %7.5f %%" % (np.mean(a) * 100.0)
else:
    print "Model predictions already produced."

Saving output to: ./dumps/
./dumps/tr_reg_080.mat , Acc: 64.96372 %
./dumps/tr_reg_081.mat , Acc: 56.77794 %
./dumps/tr_reg_082.mat , Acc: 61.50943 %
./dumps/tr_reg_083.mat , Acc: 59.28882 %
./dumps/tr_reg_084.mat , Acc: 61.24819 %
./dumps/tr_reg_085.mat , Acc: 61.77068 %
./dumps/tr_reg_086.mat , Acc: 49.49202 %
./dumps/tr_reg_087.mat , Acc: 58.37446 %
./dumps/tr_reg_088.mat , Acc: 61.53846 %
./dumps/tr_reg_089.mat , Acc: 64.80406 %
./dumps/tr_reg_090.mat , Acc: 67.92453 %
./dumps/tr_reg_091.mat , Acc: 67.41655 %
./dumps/tr_reg_092.mat , Acc: 62.67054 %
./dumps/tr_reg_093.mat , Acc: 64.81858 %
./dumps/tr_reg_094.mat , Acc: 61.24819 %
./dumps/tr_reg_095.mat , Acc: 67.35849 %
./dumps/tr_reg_096.mat , Acc: 61.07402 %
./dumps/tr_reg_097.mat , Acc: 62.49637 %
./dumps/tr_reg_098.mat , Acc: 48.92598 %
./dumps/tr_reg_099.mat , Acc: 64.44122 %

Average accuracy across all shapes: 61.40711 %


# Results

![](http://www.people.usi.ch/mascij/EG16_tutorial/shapenet_corr.png)